In [1]:
# coding:utf-8
import pandas as pd
import requests
from lxml import etree
import numpy as np
pd.set_option('display.max_columns',200)
pd.set_option('display.max_rows',400)
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import re

In [2]:
# 记录爬取内容
def write_to_file(filename,method,content):
    with open(filename,method) as f:
        f.write(content)

In [8]:
#url = 'http://www.aastocks.com/sc/ipo/ListedIPO.aspx?iid=ALL&orderby=DA&value=DESC&index={}'
def craw_ipo_list(url,page,filename,method):
    r = requests.get(url.format(page)).text
    html = etree.HTML(r)
    d = html.xpath('//td[@class="secRight"]//tr[@class="DR" or @class="ADR"]')
    for i in d:
        content = ''
        for j in i.findall('.//*'):
            if j.text.strip() !='':
                content += ('\t'+j.text.strip())
        content += '\n'
        write_to_file(filename,method,content)
   

In [9]:
# collect ipo list :2017.7.21-2019.7.21
# aastocks网站上只有最近2年的ipo名单，因此自己收集近两年的ipo名单，14-17年的名单来自github：crownpku/hk_ipo_prediction，本地名称为ipo_list.txt
# 爬取的数据直接append到ipo_list.txt中。
if __name__=='__main__':
    url = 'http://www.aastocks.com/sc/ipo/ListedIPO.aspx?iid=ALL&orderby=DA&value=DESC&index={}'
    for p in range(1,20):
        craw_ipo_list(url,p,'ipo_list.txt','a')

In [3]:
# 爬取上市当天记账货币兑港币的汇率
def get_exchange_rate(driver,date,hold,target):
    ele_date = driver.find_element_by_xpath('//*[@id="end_date_input"]')
    ele_date.click()
    ele_date.send_keys(date)
    ele_date.send_keys(Keys.ENTER)
    ele_hold = driver.find_element_by_xpath('//*[@id="quote_currency_input"]')
    ele_hold.send_keys(hold)
    ele_hold.send_keys(Keys.ENTER)
    ele_target = driver.find_element_by_xpath('//*[@id="base_currency_input"]')
    ele_target.send_keys(target)
    ele_target.send_keys(Keys.ENTER)
    time.sleep(0.5)
    ele_avg = driver.find_element_by_xpath('//*[@id="bidAskBidAvg"]')
    rate = ele_avg.text
    return(rate)

In [36]:
def str_to_f(x):
    try:
        return(float(x))
    except:
        pass

In [45]:
#　爬取基石投资者的信息：最终将各类型基石投资者的投资金额转为为：百万港币
def get_stone(date,html,driver):
    stone_type =  html.xpath('//div[@style="margin-top: 5px;"][2]//td[@style="width: 90px; padding-left: 3px"][1]//text()')
    stone_amount =  html.xpath('//div[@style="margin-top: 5px;"][2]//td[@style="width: 90px; padding-left: 3px"][2]//text()')
    n = len(stone_amount)
    firm = 0
    fund = 0
    individual = 0
    if n !=0:
        target = 'HKD'
        for i in range(n):
            a = stone_amount[i].strip()
            b = stone_type[i+1].strip()
            if a[0]=='美':
                hold = 'USD'
            elif a[0]=='R':
                hold = 'CNH'
            else:
                hold = 'HKD'       
            rate = get_exchange_rate(driver,date,hold,target)
            try:
                amount = re.match('.*?(\d+\.*\d*).*',a).group(1)
            except:
                amount = 0
            if b =='个人':
                try:
                    individual += str_to_f(rate)*str_to_f(amount)
                except:
                    pass
            elif b== '公司':
                try:
                    firm += str_to_f(rate)*str_to_f(amount)
                except:
                    pass
            else:
                try:
                    fund += str_to_f(rate)*str_to_f(amount)
                except:
                    pass
    return(firm,fund,individual)


In [39]:
def get_ipo_info(f,method,url,code,date,driver):
    r = requests.get(url.format(code)).text
    html = etree.HTML(r)
    data1 = html.xpath('.//td[@style="padding-left: 3px;"]')
    data2 = html.xpath('.//td[@style="padding-left: 3px;"]/a')
    for i in get_stone(date,html,driver):
        write_to_file(f,method,'\t'+str(i))
    for i in data1+data2:
        if i.text.strip() !='':
            write_to_file(f,method,'\t'+i.text.strip())
    write_to_file(f,method,'\n')

In [20]:
f = open('ipo_list.txt','r')
d = f.readlines()
print(d)

['date\tcode\tname\tcategory\tipo_price\tbuy_ratio\tone_hand\tdraw_prob\tfirstday_performance\tnow_price\ttotal_performance\n', '\t2017/07/20\t08465\t高科橋\t工業用品\t1.680\t13.6\t1\t100.0%\t0.00%\t1.990\t+18.45%\n', '\t2017/07/19\t08427\t瑞強集團\t電訊及網絡器材\t0.280\t11.2\t1\t100.0%\t+150.00%\t0.590\t+110.71%\n', '\t2017/07/19\t01216\t中原銀行\t銀行\t2.450\t認購不足\t1\t100.0%\t+1.63%\t2.490\t+1.63%\n', '\t2017/07/18\t08291\t萬成金屬包裝\t鋼鐵\t0.650\t12.1\t2\t90.3%\t+4.62%\t0.780\t+20.00%\n', '\t2017/07/18\t08472\t立高控股\t其他支援服務\t0.500\t10.5\t1\t100.0%\t+10.00%\t0.530\t+6.00%\n', '\t2017/07/18\t01676\t中國升海食品\t包裝食品\t0.660\t8.2\t1\t100.0%\t+100.00%\t1.310\t+98.48%\n', '\t2017/07/18\t00735\t中國電力清潔能源\t替代能源\tN/A\tN/A\tN/A\tN/A\t+11.00%\t5.030\tN/A\n', '\t2017/07/18\t01649\t內蒙古能建\t供電供熱\t1.600\t認購不足\t1\t100.0%\t-1.25%\t1.590\t-0.62%\n', '\t2017/07/17\t08481\t盛龍錦秀國際\t家具及家居用品\t0.680\t0.3\t1\t100.0%\t+42.65%\t0.620\t-8.82%\n', '\t2017/07/17\t08462\t橋英控股\t其他支援服務\t0.450\t2.2\t1\t100.0%\t+5.56%\t0.510\t+13.33%\n', '\t2017/07/17\t

In [49]:
n = 0
a ={}
for i in d:
    a[i.split('\t')[2]]=n
    n +=1

In [53]:
# 爬取2014-2019年7月ipo的股票信息（除财务信息以外
if __name__=='__main':
    url_rate = 'https://www1.oanda.com/lang/cns/currency/converter/'
    url = 'http://www.aastocks.com/sc/ipo/companysummary.aspx?symbol={}&view=1'
    header = '\tdate\tcode\tname\tcategory\tipo_price\tbuy_ratio\tone_hand\tdraw_prob\tfirstday_performance\tnow_price\ttotal_performance\tfirm\tfund\tindividual\tperiod\tpricingDate\tpublicDate\tbackDate\tipoDate\tipoArea\tindustry1\tbackgroud\tmarketArea\thand\tpriceGap\tmarketValue\thkShare\tpe1\tpredictEarning\tps1\tweb\tsponsor\teIPO\n'
    write_to_file('ipo_info.txt','a',header)
    #driver = webdriver.Chrome()
    #driver.get(url_rate)\tfirm\tfund\tindividual
    for info in d[326:]:
        i = info.split('\t')
        content = ''
        for j in i[1:]:
            content += ('\t'+j.strip())
        write_to_file('ipo_info.txt','a',content)
        code = i[2]
        date = i[1]
        get_ipo_info('ipo_info.txt','a',url,code,date,driver)
        write_to_file('ipo_list.txt','a','\n')